# Landsat 8: 
## EDS 220, Fall 2021
Friday, November 19

## Authors

- Alex Vand, UC Santa Barbara (asy@bren.ucsb.edu)
- Julia Parish, UC Santa Barbara (jparish@bren.ucsb.edu)
- Wylie Hampson, UC Santa Barbara (whampson@bren.ucsb.edu)
- Shale Hunter, UC Santa Barbara (shale@bren.ucsb.edu)

## Table of Contents

[1. Purpose](#purpose)

[2. Dataset Description](#overview)

[3. Data I/O](#io)

[4. Metadata Display and Basic Visualization](#display)

[5. Use Case Examples](#usecases)

[6. Create Binder Environment](#binder)

[7. References](#references)

<a id='purpose'></a> 
### Notebook Purpose

This notebook was created to provide an introduction to using satellite data from the <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a> sourced from Google Earth Engine for analysis to identify soil composition.


<a id='overview'></a> 
### Dataset Description


- The creators of the dataset: Joint program of USGS and NASA

- Major characteristics of the dataset: 

Data collected from 2013-04-11 to 2021-10-21.
Landsat satellites image the entire Earth every 16 days; 99 min per orbit.
CRS: WGS 84
Surface reflectance, not top of atmosphere.
Bands: 5 visible and near-infrared, 2 shortwave infrared, 1 thermal band

- The file format(s) used to store the data: TIFF

- The source/archive you will be using to retrieve the data: Google Earth Engine <a href="https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#description/" target="_blank">USGS Landsat 8 Level 2, Collection 2, Tier 1</a>

- Any known issues with data quality that might be expected to impact the results<br>
Since the data is coming from Landsat 8, we can trust that it is generally pretty reliable data. The remote sensing equipment on Landsat 8 is the Operational Land Imager (OLI) which is a multichannel scanning radiometer,and a two-channel infared radiometer, Thermal Infrared Sensor (TIRS).<br>
There are some known issues with this dataset, but many of them don’t concern most people that may be using this data. One of the most concerning issues is with cloud cover and cloud shadows. The data from this dataset can be affected if there is heavy cloud coverage. The images from the dataset have a 30 meter resolution which will allow us to get high definition images for the area of land that we will be looking at. If you were to look at a small area of land, say a few square kilometers, this data set may not be ideal because the 30 meter resolution won’t allow you to see much detail, but for this use case it works well.

<a id='io'></a> 
### Dataset Input/Output 

In [18]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWg0JzzGi8u-wLDJ3Hkk9lsIIuKboMKxiUbY6nvI6OWUFFy4VFDlhy0



Successfully saved authorization token.


<a id='display'></a> 
### Metadata Display and Basic Visualization

In [113]:
# create a centroid over region of interest, Guadalupe Mountains of New Mexico and Texas

center_lat = 32.05
center_lon = -104.74
center_point = ee.Geometry.Point(center_lon, center_lat)
roi = center_point.buffer(50000)

In [114]:
# read in Landsat 8 data from Google Earth Engine and create boundary filter using centroid point

data = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi).filter(ee.Filter.date('2021-09-01', '2021-09-30')).mean()

In [133]:
visParams = {'bands': ['SR_B1', 'SR_B5', 'SR_B7'],
             'min': 1,
             'max': 65455
            }

In [134]:
map = geemap.Map(center = [center_lat, center_lon], zoom = 7)
map.addLayer(data, visParams)
map

Map(center=[32.05, -104.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [124]:
nir = data.select('SR_B5')
swir2 = data.select('SR_B7')

oif = (nir.subtract(swir2)).divide((nir.add(swir2))).rename('OIF')

data_oif = data.addBands(oif)

In [125]:
map.addLayer(data_oif, visParams)

In [126]:
data_oif_mask = data_oif.mask('need conditional here')

<a id='usecase'></a> 
### Use Case Example

<a id='binder'></a> 
### Create Binder Environment

<a id='references'></a> 
### References

Binte Mostafiz, R.; Noguchi, R.; Ahamed, T. Agricultural Land Suitability Assessment Using Satellite Remote Sensing-Derived Soil-Vegetation Indices. Land 2021, 10, 223. https://doi.org/10.3390/land10020223

Landsat 8 data courtesy of the U.S. Geological Survey. https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#image-properties

Ridwan, M. A., Radzi, N., Ahmad, W., Mustafa, I., Din, N., Jalil, Y., Isa, A., Othman A., Zaki, W. Applications of Landsat-8 Data: Survey. Internation Journal of Engineering & Techonology 2018, 11, 436-441. DOI:10.14419/ijet.v7i4.35.22858

Vermote, E., Justice, C., Claverie, M., & Franch, B. (2016). Preliminary analysis of the performance of the Landsat 8/OLI land surface reflectance product. Remote Sensing of Environment, 185, 46-56.


